In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

/var/folders/v7/jfv0mfh910b5zdkjy1jyf02h0000gn/T/ipykernel_27969/1751419005.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [69]:
class MemoryCell:

    def __init__(self) -> None:
        self.state = np.array([1,1,1])
        self.inputs = np.array([])

    def sigmoid(self, x):
        return 1/(1 + np.exp(-x)) 
    
    def tahn(self, x):
        return np.tanh(x)
    
    def block_forget(self, x: np.array, h_previous: np.array=np.array([0])):
        # sum 
        sum_matrix = x + h_previous
        # print(f"sum: {sum_matrix}")
        result = self.sigmoid(sum_matrix)
        # print(f"result: {result}")
        # update state
        self.state = self.state * result
        print(f"state: {self.state}")

    
    def hypothesis(self, x: np.array, h_previous: np.array=np.array([0])):
        # sum 
        sum_matrix = x + h_previous
        result = self.sigmoid(sum_matrix)
        input_state = self.tahn(self.state)
        h = input_state * result
        print(f"current hypothesis: {h}")
        return h
    

    def block_input(self, x: np.array, h_previous: np.array=np.array([0])):
        sum_matrix = x + h_previous
        sig_result = self.sigmoid(sum_matrix)
        tahn_result = self.tahn(sum_matrix)
        product = sig_result * tahn_result
        # print(f"####")
        # print(f"sig_result {sig_result}")
        # print(f"tahn_result {tahn_result}")
        # print(f"product {product}")
        # print(f"####")
        self.state = self.state + product
        print(f"final memory state: {self.state}")



In [70]:
# Generate some fake data inputs

t_steps_back = 3
t_ts_features = []

t_features = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]

for i in range(t_steps_back, len(t_features) + 1):  # thanks Simone Truglia
    # print(f"{i}")
    # print(f"{t_features[t_steps_back:i]}")
    # print(f"{t_features[i - t_steps_back:i]}")
    t_ts_features.append(t_features[i - t_steps_back:i])

# print(t_ts_features)

t_ts_features = np.asarray(t_ts_features)

print(t_ts_features)

print(t_ts_features.shape)
print(t_ts_features.shape[1:])


[[ 1  2  3]
 [ 2  3  4]
 [ 3  4  5]
 [ 4  5  6]
 [ 5  6  7]
 [ 6  7  8]
 [ 7  8  9]
 [ 8  9 10]
 [ 9 10 11]
 [10 11 12]
 [11 12 13]
 [12 13 14]
 [13 14 15]
 [14 15 16]
 [15 16 17]]
(15, 3)
(3,)


In [71]:
mc = MemoryCell()

print(f"initial memory state: {mc.state}")

h_previous = np.array([0,0,0])

for i in range(len(t_ts_features)):
    print(f"time series: {i}")
    inputs = t_ts_features[i]
    s_values = mc.block_forget(inputs, h_previous)
    mc.block_input(inputs, h_previous)
    h_previous = mc.hypothesis(inputs, h_previous)
    print(f"h{i}: {h_previous}")
    print(f"###########################")


# # time 0
# inputs = np.array([5,0.1,-0.5])
# s_values = mc.block_forget(inputs, h_previous)
# mc.block_input(inputs)
# h_0 = mc.hypothesis(inputs)

# # time 1
# inputs = np.array([5,-4,-0.5])
# s_values = mc.block_forget(inputs, h_0)
# mc.block_input(inputs, h_0)
# h_1 = mc.hypothesis(inputs, h_0)

# # time 2
# inputs = np.array([5,-4,-0.5])
# s_values = mc.block_forget(inputs, h_1)
# mc.block_input(inputs, h_1)
# h_2 = mc.hypothesis(inputs, h_1)

# # time 3
# inputs = np.array([5,-8,-10])
# s_values = mc.block_forget(inputs, h_2)
# mc.block_input(inputs, h_2)
# h_3 = mc.hypothesis(inputs, h_2)


initial memory state: [1 1 1]
time series: 0
state: [0.73105858 0.88079708 0.95257413]
final memory state: [1.28782852 1.72990975 1.90043754]
current hypothesis: [0.62765529 0.82710833 0.91092273]
h0: [0.62765529 0.82710833 0.91092273]
###########################
time series: 1
state: [1.20105509 1.6930479  1.88654186]
final memory state: [2.12398996 2.67081191 2.87912232]
current hypothesis: [0.90633601 0.96936399 0.98644076]
h1: [0.90633601 0.96936399 0.98644076]
###########################
time series: 2
state: [2.08211027 2.6523843  2.87190639]
final memory state: [3.06159992 3.6453888  3.86938751]
current hypothesis: [0.97599548 0.99174711 0.99662506]
h2: [0.97599548 0.99174711 0.99662506]
###########################
time series: 3
state: [3.04061468 3.63630062 3.86585039]
final memory state: [4.03366573 4.63379509 4.86493459]
current hypothesis: [0.99252292 0.99731855 0.99896703]
h3: [0.99252292 0.99731855 0.99896703]
###########################
time series: 4
state: [4.02361733 

In [11]:
t_steps_back = 3
t_ts_features = []

t_features = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]

for i in range(t_steps_back, len(t_features) + 1):  # thanks Simone Truglia
    print(f"{i}")
    # print(f"{t_features[t_steps_back:i]}")
    # print(f"{t_features[i - t_steps_back:i]}")
    t_ts_features.append(t_features[i - t_steps_back:i])

# print(t_ts_features)

t_ts_features = np.asarray(t_ts_features)

print(t_ts_features)

print(t_ts_features.shape)
print(t_ts_features.shape[1:])

3
[1, 2, 3]
4
[2, 3, 4]
5
[3, 4, 5]
6
[4, 5, 6]
7
[5, 6, 7]
8
[6, 7, 8]
9
[7, 8, 9]
10
[8, 9, 10]
11
[9, 10, 11]
12
[10, 11, 12]
13
[11, 12, 13]
14
[12, 13, 14]
15
[13, 14, 15]
16
[14, 15, 16]
17
[15, 16, 17]
[[ 1  2  3]
 [ 2  3  4]
 [ 3  4  5]
 [ 4  5  6]
 [ 5  6  7]
 [ 6  7  8]
 [ 7  8  9]
 [ 8  9 10]
 [ 9 10 11]
 [10 11 12]
 [11 12 13]
 [12 13 14]
 [13 14 15]
 [14 15 16]
 [15 16 17]]
(15, 3)
(3,)
